#### Initial data exploration and prep for `main.py`

In [9]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd

In [10]:
load_dotenv()

True

PDF link to CTA API documentation: https://www.transitchicago.com/assets/1/6/cta_Train_Tracker_API_Developer_Guide_and_Documentation.pdf

* will use the _locations API_ to track multiple locations at once
* produces a list of in service trains w basic info
* 193 [double check] total CTA stops maybe one dict per line
    * line_name = {cta_train_stop_key_id: friendltrain_name}
* json response most important
    * ctatt: root element
    * tmst: time stamp
    * train: container element one per train in response
    * destNm: friendly destination description
    * nextStpId: next stop ID matcing GTFS
    * nextStaId: next station ID matching GTFS
    * nextStaNm: proper name of next station
* do another response request for massive list of parent stop IDs {statId:StatNm}
    * parent stop IDs are for the station overall
    * each direction at the station has it's own ID for additional detail not needed for this project
* create another dictionary that maps StatId to gio pin
* link to total number of stations per train line: https://rtams.org/services/cta/lines
* {parent_station_id: GPIO_PIN}

In [ ]:
CTA_API_KEY = os.getenv('CTA_API_KEY')

rt_names = ["Red", "Blue", "Brn", "G", "Org", "P", "Pink", "Y"]


# location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key=',
#                             CTA_API_KEY, 
#                             '&rt=', rt_names[0],
#                             '&rt=', rt_names[1],
#                             '&rt=', rt_names[2],
#                             '&rt=', rt_names[3],
#                             '&outputType=JSON'])


# location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
#                                 ,CTA_API_KEY
#                                 ,'&rt=', rt_names[-4]
#                                 ,'&rt=', rt_names[-3]
#                                 ,'&rt=', rt_names[-2]
#                                 ,'&rt=', rt_names[-1]
#                                 ,'&outputType=JSON'])


# response = requests.get(location_url).json()

print(response)

locations_df = pd.DataFrame()

df_cols = ['next_station_parent_id', 'train_line_name', 'next_station_name', 'next_stop_id', 
           'arrival_time', 'prediction_time', 'train_direction', 'is_delayed' ]

stop = 0
train = 0



all_next_sta_p_id = []
all_trn_ln_nm = []
all_nxt_sta_names = []
all_nxt_stop_id = []
all_arrival_times = []
all_pred_times = []
trn_dirctns = []
all_is_dlyd = []


# list of lists to build df
all_lapi_trn_data = []
all_nxt_sta_dict= {}

# rt_len = len(response['ctatt']['route'])
# for train in range(rt_len):
#         for stop in range(len(response['ctatt']['route'][train]['train'])):
#                 all_nxt_sta_dict[response['ctatt']['route'][train]['train'][stop]['nextStaId']] = response['ctatt']['route'][train]['train'][stop]['nextStaNm']
#                  # repeat this line a lot maybe build funtion for it
#                 all_nxt_sta_names.append(response['ctatt']['route'][train]['train'][stop]['nextStaNm'])
#                 stop+=1
#         train+=1

# print(all_nxt_sta_dict, '\n', all_nxt_sta_names)



{'ctatt': {'tmst': '2026-02-26T21:58:08', 'errCd': '0', 'errNm': None, 'route': [{'@name': 'org', 'train': [{'rn': '717', 'destSt': '30182', 'destNm': 'Midway', 'trDr': '5', 'nextStaId': '40930', 'nextStpId': '30182', 'nextStaNm': 'Midway', 'prdt': '2026-02-26T21:57:57', 'arrT': '2026-02-26T21:58:57', 'isApp': '1', 'isDly': '0', 'flags': None, 'lat': '41.79788', 'lon': '-87.72957', 'heading': '248'}, {'rn': '718', 'destSt': '30182', 'destNm': 'Midway', 'trDr': '5', 'nextStaId': '41130', 'nextStpId': '30216', 'nextStaNm': 'Halsted', 'prdt': '2026-02-26T21:58:03', 'arrT': '2026-02-26T22:00:03', 'isApp': '0', 'isDly': '0', 'flags': None, 'lat': '41.85761', 'lon': '-87.63106', 'heading': '265'}, {'rn': '719', 'destSt': '30182', 'destNm': 'Loop', 'trDr': '1', 'nextStaId': '41060', 'nextStpId': '30205', 'nextStaNm': 'Ashland', 'prdt': '2026-02-26T21:57:36', 'arrT': '2026-02-26T21:59:36', 'isApp': '0', 'isDly': '0', 'flags': None, 'lat': '41.83076', 'lon': '-87.67865', 'heading': '58'}, {'rn'

In [19]:
# first 0 is train second 0 is stop
print(response['ctatt']['route'][0]['train'][0])

{'rn': '717', 'destSt': '30182', 'destNm': 'Midway', 'trDr': '5', 'nextStaId': '40930', 'nextStpId': '30182', 'nextStaNm': 'Midway', 'prdt': '2026-02-26T21:57:57', 'arrT': '2026-02-26T21:58:57', 'isApp': '1', 'isDly': '0', 'flags': None, 'lat': '41.79788', 'lon': '-87.72957', 'heading': '248'}


need to reformat how I get data

Goal:
* set up local postgres database to store train tracker info
* two dataframes
    * one that will have _all_ station ids and station names
    * second for storing info from the api
    * key of dictionary is col name value is array that will populate column
        * `dict(zip(my_list_1, my_list_2))`
        * stop id
            * not parent id
            * ex 30219 
            * join on stop id since stop id is unique and parent id is not
            * `response['ctatt']['route'][0]['train'][0]['nextStpId']`
        * train line name
            * useful for filtering queries
            * have each train line map to an ID and store ID to train @name in seperate db table?
            * `response['ctatt']['route'][0]['@name']`
        * next station id
            * parent stop id [non directional]
            * `response['ctatt']['route'][0]['train'][0]['nextStaId']`
        * next station name
            * ex. Kedzie
            * `response['ctatt']['route'][0]['train'][0]['nextStaNm']`
        * station arrival time
            * prediction of train arrival time
            * `response['ctatt']['route'][0]['train'][0]['arrT']`
            * ex. 2026-02-26T19:35:19
        * station prediction time
            * timestamp for when _prediction_ was generated
            * `response['ctatt']['route'][0]['train'][0]['prdt']`
            * ex. 2026-02-26T19:33:19
        * station TrDr
            * numeric code that translates to direction
            * ex. for red line 1 is howard bound 5 is 95th bound
            * codes will only be 1 **or** 5
            * `response['ctatt']['route'][0]['train'][0]['trDr']`
        * is station delayed
            * bool to denote if tain is delayed or not
            * `response['ctatt']['route'][0]['train'][0]['isDly']`
        * after df is built and clean dtypes load it into local postgres db
            * `pd.to_sql("table_name", if_exists=append, index=False, con=con)`
            * `conn = psycopg2.connect(
                dbname="school",
                user="postgres",
                password="your_password",
                host="localhost")`
                * https://www.geeksforgeeks.org/postgresql/postgresql-connecting-to-the-database-using-python/
---
* create CTA database w the following schemas [tables]
    1. location
        * cols: stop id, parent station id, train line id, next station name, prediction time, arrival time, is delayed, direction id, 
        * primary key: unique location id when table writes
        * foreign keys: stop id, direction id, parent station id, train line id
        * train_line_name ex. org, rt names
    2. parent station
        * cols: parent station id, parent station name
        * primary key: parent station id
        * foreign keys: none?
        * lapi table will have direction info
    3. non-parent station / stop station
        * if station names differ from parent and non parent station names then keep two tables if not only keep this table
        * cols: stop station name, stop id, parent station id
         * primary key: non parent station / stop id
         * foreign key: parent station id
    4. train direction
        * cols: train line id, train line name, direction id
        * primary key: unique direction id
            * create own direction ids to not repeat 1 and 5?
                * would need to clean/update df before loading to table w nice ids
        * foreign key: train line id
    5. train line name
        * cols: train line id, train line name
        * primary key: train line id
        * foreign key: none?
            * for example 1 for red, 2 for blue, 3 for green, etc.
    

    * general flow
        * location, parent station, non-parent station / stop station, train direction, train line name tables 

In [17]:
my_col_name = ['train']
my_data = [['orange', 'red', 'blue']]
my_dict = dict(zip(my_col_name, my_data))
my_df = pd.DataFrame(my_dict)

my_df.head()

,train
0,orange
1,red
2,blue


In [8]:
my_test_1 =[1, 2, 3]
my_test_2 = [2, 3, 4]

combine_lists = []
combine_lists.append(my_test_2)
combine_lists.append(my_test_1)

print(combine_lists)

[[2, 3, 4], [1, 2, 3]]


In [ ]:
def get_half_trns_lapi_call(is_first_half_rt_nms: bool) -> dict[int, str]:

    CTA_API_KEY = os.getenv('CTA_API_KEY')

    rt_names = ["Red", "Blue", "Brn", "G", "Org", "P", "Pink", "Y"]

    
    if is_first_half_rt_nms==1: 
        location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
                                ,CTA_API_KEY
                                ,'&rt=', rt_names[0]
                                ,'&rt=', rt_names[1]
                                ,'&rt=', rt_names[2]    # api has max of four train lines at once
                                ,'&rt=', rt_names[3]
                                ,'&outputType=JSON'])

    else:
        location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
                                ,CTA_API_KEY
                                ,'&rt=', rt_names[-4]
                                ,'&rt=', rt_names[-3]
                                ,'&rt=', rt_names[-2]
                                ,'&rt=', rt_names[-1]
                                ,'&outputType=JSON'])
        

    response = requests.get(location_url).json()


    stop = 0
    train = 0
    rt_len = len(response['ctatt']['route'])
    all_nxt_sta_dict= {} # {stationId: station_name}


    for train in range(rt_len):
        
        for stop in range(len(response['ctatt']['route'][train]['train'])): # active stops on train line
                all_nxt_sta_dict[int(response['ctatt']['route'][train]['train'][stop]['nextStaId'])] = response['ctatt']['route'][train]['train'][stop]['nextStaNm']
                stop+=1

        train+=1

        
    return all_nxt_sta_dict

In [53]:
get_half_trns_lapi_call(is_first_half_rt_lst=False)

{40310: 'Western',
 41130: 'Halsted',
 40040: 'Quincy',
 40930: 'Midway',
 40960: 'Pulaski',
 40900: 'Howard',
 40400: 'Noyes',
 40740: 'Western',
 40170: 'Ashland',
 41030: 'Polk',
 40850: 'Harold Washington Library-State/Van Buren',
 41680: 'Oakton-Skokie'}

In [54]:
get_half_trns_lapi_call(is_first_half_rt_lst=True)

{41000: 'Cermak-Chinatown',
 40450: '95th/Dan Ryan',
 41490: 'Harrison',
 41420: 'Addison',
 41170: 'Garfield',
 41300: 'Loyola',
 40650: 'North/Clybourn',
 41220: 'Fullerton',
 41400: 'Roosevelt',
 40080: 'Sheridan',
 40760: 'Granville',
 41320: 'Belmont',
 41450: 'Chicago',
 40910: '63rd',
 40470: 'Racine',
 40010: 'Austin',
 40250: 'Kedzie-Homan',
 40670: "Western (O'Hare Branch)",
 40750: "Harlem (O'Hare Branch)",
 41330: 'Montrose',
 41410: 'Chicago',
 41240: 'Addison',
 40980: 'Harlem (Forest Park Branch)',
 40810: 'Illinois Medical District',
 40800: 'Sedgwick',
 40660: 'Armitage',
 40680: 'Adams/Wabash',
 40090: 'Damen',
 40360: 'Southport',
 40710: 'Chicago',
 41480: 'Western',
 41700: 'Washington/Wabash',
 40020: 'Harlem/Lake',
 40290: 'Ashland/63rd',
 40130: '51st',
 41710: 'Damen',
 41120: '35th-Bronzeville-IIT',
 40480: 'Cicero',
 40700: 'Laramie',
 40170: 'Ashland'}

* need to do another API call or just download file [explore + research pros and cons] to get all CTA station stops to map to all GPIO pins
* probably only do call once when running map and store data but continue to run `get_half_trns_lapi_call` to get up to date train location info

In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# `https://data.cityofchicago.org/api/v3/views/8pix-ypme/query.json`
https://data.cityofchicago.org/resource/8pix-ypme.json
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofchicago.org,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("8pix-ypme", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
